In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import hdf5storage
import glob
import os

In [ ]:
raw_files_base_dir = '/kucresis/scratch/data/UTIG/UTIG2/orig/xlob/'
tmp_hdr_files_base_dir = '/kucresis/scratch/dataproducts/opr_data/opr_tmp/headers/rds/2017_Antarctica_BaslerJKB/'
tmp_hdr_glob = tmp_hdr_files_base_dir + '**/bxds.mat'

for filepath in glob.glob(tmp_hdr_glob, recursive = True):
    tmp_hdr = hdf5storage.loadmat(filepath)

    raw_file_path = os.path.join(raw_files_base_dir, os.path.relpath(filepath, tmp_hdr_files_base_dir)[:-4])
    raw_file_size = os.path.getsize(raw_file_path)

    max_offset = np.max(tmp_hdr['offset'])

    difference = raw_file_size - max_offset

    if difference != 6400:
        print(f"[{filepath}] Max offset: {max_offset}, \tRaw file size: {raw_file_size}, \tDifference: {difference}")

        if difference < 6400:
            print(f"  ==> WARNING: Offset would point to bytes outside of the file.")


In [ ]:
tmp_hdr_glob

In [ ]:
#tmp_hdr_fn = '/kucresis/scratch/tteisberg_sta/scripts/opr_user_tmp/headers/rds/2017_Antarctica_BaslerJKB/KNX/JKB2s/X100b/RADnh5/bxds.mat'
#tmp_hdr_fn = '/kucresis/scratch/dataproducts/opr_data/opr_tmp/headers/rds/2017_Antarctica_BaslerJKB/KNX/JKB2s/X100a/RADnh5/bxds.mat'
tmp_hdr_fn = '/kucresis/scratch/dataproducts/opr_data/opr_tmp/headers/rds/2017_Antarctica_BaslerJKB/ASB/JKB2s/GL0100c/RADnh5/bxds.mat'

print(f"Loading temporary header file: {tmp_hdr_fn}")
print(f"File exists: {Path(tmp_hdr_fn).exists()}")

tmp_hdr = hdf5storage.loadmat(tmp_hdr_fn)
print(f"Keys in temporary header file: {list(tmp_hdr.keys())}")

In [ ]:
for k in tmp_hdr.keys():
    print(f"== {k} ==")
    print(f"  Type: {tmp_hdr[k].dtype}")
    print(f"  Shape: {tmp_hdr[k].shape}")
    # Check if is numeric before min/max
    if np.issubdtype(tmp_hdr[k].dtype, np.number):
        print(f"  Min: {np.nanmin(tmp_hdr[k])}")
        print(f"  Max: {np.nanmax(tmp_hdr[k])}")
        print(f"  # NaNs: {np.sum(np.isnan(tmp_hdr[k]))} out of {tmp_hdr[k].size}")

In [ ]:
tmp_hdr['offset']

In [ ]:
# Convert comp_time to datetime as seconds since unix epoch
unix_epoch = np.datetime64('1970-01-01T00:00:00')
comp_time = unix_epoch + np.array(tmp_hdr['comp_time'], dtype='timedelta64[s]')
print(f"comp_time range: {comp_time[0]} to {comp_time[-1]} ({comp_time[-1] - comp_time[0]})")

radar_time = np.array((tmp_hdr['radar_time']), dtype=np.float64).squeeze()
print(f"radar_time range: {radar_time[0]} to {radar_time[-1]} ({radar_time[-1] - radar_time[0]})")

In [ ]:
comp_time.shape

In [ ]:
fig, axs = plt.subplots(2,1, figsize=(10,8), sharex=True)
comp_time_deltas = np.diff(tmp_hdr['comp_time'])
axs[0].plot(comp_time_deltas, label='comp_time_deltas')
axs[0].set_ylim(np.minimum(0, 1.1*np.min(comp_time_deltas)), np.max(comp_time_deltas) * 1.1)

radar_time_deltas = np.diff(radar_time)
axs[1].plot(radar_time_deltas, label='radar_time_deltas')
axs[1].set_ylim(np.minimum(0, 1.1*np.min(radar_time_deltas)), np.max(radar_time_deltas) * 1.1)

#axs[1].set_xlim(0, 2000)